In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import scsavailability as scs
    
import statsmodels.api as sm
from scipy import stats

from scsavailability import features as feat, model as md, plotting as pt, score as sc


2020-12-21 14:18:44,487 - NumExpr defaulting to 8 threads.


In [2]:
path = 'C:/Users/Jamie.williams/OneDrive - Newton Europe Ltd/Castle Donnington/Data/'

In [3]:
at = pd.read_csv(path+'active_totes_20201210.csv')
av = pd.read_csv(path+'Availability_with_Grey&Blue_1811-0912.csv',names = ["timestamp","Pick Station","Availability","Blue Tote Loss","Grey Tote Loss"])
#fa = pd.read_csv(path + 'scs_tote_matched.csv')
scs_raw = pd.read_csv(path + 'Faults20_11-10_12.csv')

In [4]:
fa = feat.add_code(scs_raw)
fa, unmapped = feat.add_tote_colour(fa)

Running: Function "load_tote_lookup" (id=2712497770848) was called 1 times


In [5]:
at = feat.pre_process_AT(at)

Running: Function "pre_process_AT" (id=2712497772576) was called 1 times


In [6]:
av = feat.pre_process_av(av)

Running: Function "pre_process_av" (id=2712497773440) was called 1 times


In [7]:
fa = feat.preprocess_faults(fa,remove_same_location_faults = True)

Running: Function "preprocess_faults" (id=2712497772432) was called 1 times
Running: Function "load_module_lookup" (id=2712497774160) was called 1 times
Running: Function "load_ID_lookup" (id=2712497772864) was called 1 times
duplicated location faults removed - max duration kept
HOTFIX: Quadrant only faults, PTT Asset Code update


In [8]:
fa_floor = feat.floor_shift_time_fa(fa, shift=15)

Time shifted by 15Minutes


In [9]:
fa_sel = feat.fault_select(fa_floor, fault_select_options = {'Tote Colour':['Both','Blue','Grey']})
                                                            
fa_agg = feat.faults_aggregate(fa_sel,fault_agg_level= 'Asset Code', agg_type = 'count')

In [10]:
av_sel,at_sel = feat.av_at_select(av, at, availability_select_options = None, remove_high_AT = True, AT_limit = None)

av_agg = feat.aggregate_availability(av_sel, agg_level ='None')
at_agg = feat.aggregate_totes(at_sel, agg_level = 'None')

#at_agg = at_agg[at_agg['TOTES']>100]

Running: Function "aggregate_availability" (id=2712497772288) was called 1 times


In [11]:
df = feat.merge_av_fa_at(av_agg ,at_df=at_agg, fa_df = fa_agg , agg_level = 'None')

In [12]:
X,y = md.gen_feat_var(df,target = 'Blue Tote Loss', features = ['Totes'])
y=y

In [13]:
X_train, X_test, y_train, y_test = md.split(X,y,split_options = {'test_size': 0.3,
                                                                 'random_state': None})

RF_mdl,predictions_RF=md.run_RF_model(X_train, X_test, y_train, y_test,RF_options = {'num_trees': 100, 
                                                                                    'criterion': 'mse', 
                                                                                    'max_depth': None, 
                                                                                    'dtree': False})

Linear_mdl,predictions_LM,Coeff,fit_metrics = md.run_LR_model(X_train, X_test, y_train, y_test)

XGB_mdl,predictions_XGB,Importance_XGB = md.run_XGB_model(X_train, X_test, y_train, y_test, XGB_options = {'num_trees': 100,
                                                                                                         'max_depth': None})

selection_options= {'threshold': -np.inf,
                    'max_features': None}

X_sel_rf = md.select_features(X, y, RF_mdl, **selection_options)

X_sel_lm = md.select_features(X, y, Linear_mdl, **selection_options)

#X_sel_XGB = md.select_features(X, y, XGB_mdl, selection_options)

X_train, X_test, y_train, y_test = md.split(X_sel_lm,y,split_options = {'test_size': 0.3,
                                                                       'random_state': None})

Linear_mdl,predictions_LM,Coeff,fit_metrics = md.run_LR_model(X_train, X_test, y_train, y_test)

cv_R2 = md.cross_validate_r2(Linear_mdl, X, y)

cv_R2 = md.cross_validate_r2(RF_mdl,X, y)

cv_R2 = md.cross_validate_r2(XGB_mdl, X, y)

Feature Importance Ranking: 
 
    Importance Feature
0         1.0   TOTES

Regression Metrics: 
 
           RF Metrics
MAE         0.044797
MSE         0.003393
RMSE        0.058252
MAPE%      20.450000
ACC%       79.550000
OOB         0.696337
R2_Train    0.956736
R2_Pred     0.818279 

Feature Importance Ranking: 
 
    Importance Feature
0         1.0   TOTES

Regression Metrics: 
 
          XGB Metrics
MAE        0.0522889
MSE       0.00496242
RMSE       0.0704444
MAPE%          23.47
ACC%           76.53
OOB              N/A
R2_Train    0.990754
R2_Pred     0.734252 


Number of Selected Features:1 


Number of Selected Features:1 


Cross Validation Scores LinearRegression(): 
 
        R2 Scores
1       0.827117
2       0.870480
3       0.863587
4       0.801619
5       0.894973
6       0.772764
7       0.883703
8       0.593837
9       0.776309
10      0.806958
Mean    0.809135
STD     0.082943

Cross Validation Scores RandomForestRegressor(oob_score=True): 
 
        R2 Sc

### Add LM - metrics

In [14]:
sc.lm_coefficients(Linear_mdl, X_test ,y_test)

C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1847: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= np.asarray(_b)) & cond0


,Feature,Coefficients,Standard Errors,t values,Probabilities
0,Constant,0.7177,NaN,NaN,NaN
1,TOTES,-0.0108,NaN,NaN,NaN


In [15]:
X_train = sm.add_constant(X_train)
model = sm.OLS(y_train,X_train)
results = model.fit()
print(results.summary())

X_new = sm.add_constant(X_test)
#y_prediction =  results.predict(X_test)

# plt.scatter(y_prediction,y_test)

# from sklearn.metrics import r2_score
# print(r2_score(y_prediction,y_test))

                            OLS Regression Results                            
Dep. Variable:         Blue Tote Loss   R-squared:                       0.830
Model:                            OLS   Adj. R-squared:                  0.830
Method:                 Least Squares   F-statistic:                     1612.
Date:                Mon, 21 Dec 2020   Prob (F-statistic):          7.69e-129
Time:                        14:19:39   Log-Likelihood:                 519.31
No. Observations:                 331   AIC:                            -1035.
Df Residuals:                     329   BIC:                            -1027.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.7177      0.012     60.298      0.0

In [16]:
df_greyhigh_ID = df.drop(['Grey Tote Loss','TOTES','timestamp'],axis=1)

df_greyhigh_ID.sum().sort_values(ascending=False).head(20)

C45            4765.0
C41            4367.0
C40            3834.0
C49            3374.0
C37            2472.0
C08PTT082      2445.0
C2301RDC061    2372.0
C1703STA191    2051.0
SCS222         1892.0
C0602RDC036    1604.0
C12PTT151      1599.0
C14PTT202      1554.0
C44            1503.0
C14PTT201      1408.0
SCSM18         1396.0
SCSM22         1391.0
C09PTT092      1347.0
C09PTT102      1290.0
SCS181         1232.0
C06PTT031      1104.0
dtype: float64

In [17]:
df_grey_ID = df.drop(['Grey Tote Loss','TOTES','timestamp'],axis=1)

df_grey_ID.sum().sort_values(ascending=False).head(20)

C45            4765.0
C41            4367.0
C40            3834.0
C49            3374.0
C37            2472.0
C08PTT082      2445.0
C2301RDC061    2372.0
C1703STA191    2051.0
SCS222         1892.0
C0602RDC036    1604.0
C12PTT151      1599.0
C14PTT202      1554.0
C44            1503.0
C14PTT201      1408.0
SCSM18         1396.0
SCSM22         1391.0
C09PTT092      1347.0
C09PTT102      1290.0
SCS181         1232.0
C06PTT031      1104.0
dtype: float64

In [18]:
df_greyhigh_Asset = df.drop(['Grey Tote Loss','TOTES','timestamp'],axis=1)

df_greyhigh_Asset.sum().sort_values(ascending=False).head(20)

C45            4765.0
C41            4367.0
C40            3834.0
C49            3374.0
C37            2472.0
C08PTT082      2445.0
C2301RDC061    2372.0
C1703STA191    2051.0
SCS222         1892.0
C0602RDC036    1604.0
C12PTT151      1599.0
C14PTT202      1554.0
C44            1503.0
C14PTT201      1408.0
SCSM18         1396.0
SCSM22         1391.0
C09PTT092      1347.0
C09PTT102      1290.0
SCS181         1232.0
C06PTT031      1104.0
dtype: float64

In [19]:
df_grey_Asset = df.drop(['Grey Tote Loss','TOTES','timestamp'],axis=1)

df_grey_Asset.sum().sort_values(ascending=False).head(20)

C45            4765.0
C41            4367.0
C40            3834.0
C49            3374.0
C37            2472.0
C08PTT082      2445.0
C2301RDC061    2372.0
C1703STA191    2051.0
SCS222         1892.0
C0602RDC036    1604.0
C12PTT151      1599.0
C14PTT202      1554.0
C44            1503.0
C14PTT201      1408.0
SCSM18         1396.0
SCSM22         1391.0
C09PTT092      1347.0
C09PTT102      1290.0
SCS181         1232.0
C06PTT031      1104.0
dtype: float64

In [20]:
av_agg

,Downtime,Blue Tote Loss,Grey Tote Loss
timestamp,,,
2020-10-16 06:00:00,0.381125,NaN,NaN
2020-10-16 07:00:00,0.327088,NaN,NaN
2020-10-16 08:00:00,0.234701,NaN,NaN
2020-10-16 09:00:00,0.168205,NaN,NaN
2020-10-16 10:00:00,0.231875,NaN,NaN
...,...,...,...
2020-12-09 13:00:00,0.241812,0.165028,0.076784
2020-12-09 14:00:00,0.226444,0.123108,0.103336
2020-12-09 15:00:00,0.277924,0.143318,0.134606


In [21]:
y

0      0.091797
1      0.092953
2      0.298307
3      0.103937
4      0.331421
         ...   
469    0.165028
470    0.123108
471    0.143318
472    0.064058
473    0.204613
Name: Blue Tote Loss, Length: 474, dtype: float64

In [22]:
fa

,Number,Alert,timestamp,PLC,Desk,Duration,Fault ID,Asset Code,Tote Colour,Loop,Suffix,Pick Station,PLCN,Quadrant,MODULE,Alert Type
48430,50586.0,C06: C0602RDC036 036AQ2 roller motor unit fault,2020-12-01 05:08:28,C06,P02,1.0,3429.0,C0602RDC036,Blue,Module,C0602,False,6.0,1.0,2,Fault
45111,186939.0,C23: C2301RDC061 061AQ1 roller motor unit fault,2020-11-20 13:51:55,C23,P01,1.0,3429.0,C2301RDC061,Blue,Outside,C2301,False,23.0,0.0,NaN,Fault
160099,419745.0,SCS121: main switch is OFF in automaticmode X:...,2020-12-04 16:04:05,SCSM12,SCS121,1.0,5256.0,SCS121,Grey,carousel,SCSM12,False,0.0,3.0,12,Fault
45112,186939.0,C23: C2301RDC061 061AQ1 roller motor unit fault,2020-11-20 13:51:39,C23,P01,1.0,3429.0,C2301RDC061,Blue,Outside,C2301,False,23.0,0.0,NaN,Fault
91301,501021.0,C37: door contact switch: S21,2020-12-06 07:54:23,C37,Z,1.0,0.0,C37,Blue,NaN,NaN,False,37.0,0.0,NaN,Fault
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168380,127995.0,C15: C15+FD1 Fire Alarm system alarm,2020-11-20 10:32:20,C15,Z,75917.0,3242.0,C15,Blue,NaN,NaN,False,15.0,0.0,NaN,Fault
48767,188777.0,C23: C2303RDC194 194AQ1 roller motor unit fault,2020-12-06 08:03:18,C23,P03,76318.0,3429.0,C2303RDC194,Blue,Outside,C2303,False,23.0,0.0,NaN,Fault
123485,186559.0,C23: C2303BCL038 038QM11 motor circuit breaker...,2020-12-06 08:03:15,C23,P03,76322.0,3430.0,C2303BCL038,Blue,Outside,C2303,False,23.0,0.0,NaN,Fault
102601,141205.0,C17: C1701RDC251 251B11 occupancy sensor fault,2020-11-20 07:34:23,C17,P01,84265.0,3416.0,C1701RDC251,Blue,NaN,NaN,False,17.0,0.0,NaN,Fault


In [23]:
fa[((fa['Alert'].str.contains('access door')) & (fa['Duration']>3600))]

,Number,Alert,timestamp,PLC,Desk,Duration,Fault ID,Asset Code,Tote Colour,Loop,Suffix,Pick Station,PLCN,Quadrant,MODULE,Alert Type
48430,50586.0,C06: C0602RDC036 036AQ2 roller motor unit fault,2020-12-01 05:08:28,C06,P02,1.0,3429.0,C0602RDC036,Blue,Module,C0602,False,6.0,1.0,2,Fault
45111,186939.0,C23: C2301RDC061 061AQ1 roller motor unit fault,2020-11-20 13:51:55,C23,P01,1.0,3429.0,C2301RDC061,Blue,Outside,C2301,False,23.0,0.0,NaN,Fault
160099,419745.0,SCS121: main switch is OFF in automaticmode X:...,2020-12-04 16:04:05,SCSM12,SCS121,1.0,5256.0,SCS121,Grey,carousel,SCSM12,False,0.0,3.0,12,Fault
45112,186939.0,C23: C2301RDC061 061AQ1 roller motor unit fault,2020-11-20 13:51:39,C23,P01,1.0,3429.0,C2301RDC061,Blue,Outside,C2301,False,23.0,0.0,NaN,Fault
91301,501021.0,C37: door contact switch: S21,2020-12-06 07:54:23,C37,Z,1.0,0.0,C37,Blue,NaN,NaN,False,37.0,0.0,NaN,Fault
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168380,127995.0,C15: C15+FD1 Fire Alarm system alarm,2020-11-20 10:32:20,C15,Z,75917.0,3242.0,C15,Blue,NaN,NaN,False,15.0,0.0,NaN,Fault
48767,188777.0,C23: C2303RDC194 194AQ1 roller motor unit fault,2020-12-06 08:03:18,C23,P03,76318.0,3429.0,C2303RDC194,Blue,Outside,C2303,False,23.0,0.0,NaN,Fault
123485,186559.0,C23: C2303BCL038 038QM11 motor circuit breaker...,2020-12-06 08:03:15,C23,P03,76322.0,3430.0,C2303BCL038,Blue,Outside,C2303,False,23.0,0.0,NaN,Fault
102601,141205.0,C17: C1701RDC251 251B11 occupancy sensor fault,2020-11-20 07:34:23,C17,P01,84265.0,3416.0,C1701RDC251,Blue,NaN,NaN,False,17.0,0.0,NaN,Fault
